In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json

def costruisci_excel(anno_inizio):
    matrice = []

    def appendi(array, http, giornata):
        re = requests.get(http)
        soup = BeautifulSoup(re.text, "html.parser")
        for link in soup.find_all("div", attrs={"class": "box-partita"}):
            vinta_a_tavolino = link.find("div", attrs={"class": "dcr"})
            risultato_sul_campo = link.find("span", attrs={"class": "note"})
            data = link.div.span.string.split(" ")[0]
            casa = link.find("div", attrs={"class": "col-xs-6 risultatosx"})
            ospite = link.find("div", attrs={"class": "col-xs-6 risultatodx"})
            squadra_casa = casa.h4.string.split(" ")[-1]
            squadra_ospite = ospite.h4.string.split(" ")[-1]
            if risultato_sul_campo is not None and risultato_sul_campo.string not in ["Partita non disputata", None]:
                gol_casa, gol_ospite = risultato_sul_campo.string.split(" ")[-1].split("-")
            elif vinta_a_tavolino is None:
                gol_casa = casa.span.string
                gol_ospite = ospite.span.string
            else:
                gol_casa, gol_ospite = "-", "-"
            if (gol_casa, gol_ospite) != ("-", "-"):
                        array.append([giornata, data, squadra_casa, squadra_ospite, gol_casa, gol_ospite])

    for i in range(38):
        giornata = i+1
        appendi(matrice, f"http://www.legaseriea.it/it/serie-a/calendario-e-risultati/20"
                         f"{anno_inizio}-{anno_inizio + 1}/UNICO/UNI/{giornata}", giornata)

    dataframe = pd.DataFrame(np.array(matrice), columns=["Giornata", "Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG"])
    path = f"season/season-{anno_inizio}{anno_inizio + 1}_csv"
    dataframe.to_excel(f"{path}.xlsx", index=False)
    return f"{path}.xlsx"
    
def leggi(anno_inizio, create=True):
    path = f"season/season-{anno_inizio}{anno_inizio + 1}_csv"
    if not create:
        try:
            return pd.read_excel(f"{path}.xlsx")
        except:
            print(f"{path}.xlsx not exists\ncreating it")
    return pd.read_excel(costruisci_excel(anno_inizio))